<a href="https://colab.research.google.com/github/do71730/Learning/blob/main/milestone3_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIVOC-91 INTERFACE

App will close connect after executing a function, from user. if you want to see how the database change after running the app run the sql magic cells after changing the table name.To delete the database, close colab.

In [ ]:
# tableschema = 'dovi'
import  sqlite3
conn  =  sqlite3 .connect( 'mydatabase.db' )
cursor  =  conn.cursor ()

cursor.executescript("""

    drop table if exists dispatch;
    drop table if exists modification;
    drop table if exists vaccine_order;
    drop table if exists chief;
    drop table if exists staff;
    drop table if exists Registration;
    drop table if exists residency;
    drop table if exists folks;
    drop table if exists health_center;
    drop table if exists places;
    drop trigger if exists update_staff;
    drop trigger if exists update_folks;
    drop trigger if exists update_place;
    drop trigger if exists update_health_center;
    drop trigger if exists new_registration;
    drop trigger if exists insert_mod;

    create table if not exists folks (
    PIN bigint not null check(PIN >= 1000000000000000 and PIN <= 9999999999999999),
    first_name varchar(25),
    last_name varchar(25),
    DOB date,
    work_phone bigint check (work_phone >= 1000000000 and work_phone <= 9999999999),
    mobile_phone bigint check (mobile_phone >= 1000000000 and mobile_phone <= 9999999999),
    home_phone bigint check (home_phone >= 1000000000 and home_phone <= 9999999999),
    email json,
    primary key(PIN)
    );


    create table if not exists staff(
    S_PIN bigint not null check(S_PIN >= 1000000000000000 and S_PIN <= 9999999999999999),
    primary key(S_PIN),
    foreign key (S_PIN) references folks(PIN) ON DELETE CASCADE
    );

    create table if not exists chief(
    C_PIN bigint not null check(C_PIN >= 1000000000000000 and C_PIN <= 9999999999999999),
    primary key(C_PIN),
    foreign key (C_PIN) references staff(S_PIN) ON DELETE CASCADE
    );

    /* all the places*/
    create table if not exists places(
    street_number integer not null,
    name varchar(25) not null ,
    city varchar(25) not null,
    state varchar(25) not null,
    zipCode int not null, 
    x_cord int not null,
    y_cord int not null,
    primary key(street_number,name,city,state,zipCode),
    unique(x_cord,y_cord)
    );


    create table if not exists residency(
    x_cord int not null,
    y_cord int not null,
    PIN bigint not null check(PIN >= 1000000000000000 and PIN <= 9999999999999999),
    foreign key(x_cord,y_cord) references places (x_cord,y_cord) ON DELETE CASCADE,
    foreign key (PIN) references folks(PIN) ON DELETE CASCADE
    );


    create table if not exists health_center (
    hc_date date not null,
    x_cord int not null,
    y_cord int not null,
    max_capacity int not null,
    curr_size int,
    constraint size check (curr_size <= max_capacity),
    primary key(hc_date,x_cord,y_cord),
    foreign key(x_cord,y_cord) references places (x_cord,y_cord) ON DELETE CASCADE);


    create table if not exists registration(
    PIN bigint not null check(PIN >= 1000000000000000 and PIN <= 9999999999999999),
    r_date date not null,
    x_cord int not null,
    y_cord int not null,
    primary key(PIN, r_date),
    foreign key(r_date,x_cord,y_cord) references health_center (hc_date,x_cord,y_cord)  ON DELETE CASCADE,
    foreign key (PIN) references folks(PIN) ON DELETE CASCADE
    );

    create table if not exists dispatch(
    dDay date not null,
    start_time time not null,
    end_time time not null,
    S_PIN bigint not null check(S_PIN >= 1000000000000000 and S_PIN <= 9999999999999999),
    x_cord int not null,
    y_cord int not null,
    C_PIN bigint not null check(C_PIN >= 1000000000000000 and C_PIN <= 9999999999999999),
    primary key(dDay,start_time,end_time,x_cord,y_cord,S_PIN),
    foreign key(dDay,x_cord,y_cord) references health_center (hc_date,x_cord,y_cord)  ON DELETE CASCADE,
    foreign key (S_PIN) references staff(S_PIN) ON DELETE CASCADE,
    foreign key (C_PIN) references chief(C_PIN) ON DELETE CASCADE
    );

    create table if not exists vaccine_order (
    PIN bigint not null check(PIN >= 1000000000000000 and PIN <= 9999999999999999),
    order_num int not null,
    num_vacinne int not null,
    order_date datetime not null,
    delivery_date date not null,
    x_cord int not null,
    y_cord int not null,
    primary key(order_num),
    foreign key (PIN) references staff(S_PIN) ON DELETE CASCADE,
    foreign key(delivery_date,x_cord,y_cord) references health_center (hc_date,x_cord,y_cord) ON DELETE CASCADE );

    create table if not exists modification (
    mod_timestamp datetime,
    m_PIN bigint check(m_PIN >= 1000000000000000 and m_PIN <= 9999999999999999),
    order_num int not null,
    foreign key (m_PIN) references staff(S_PIN) ON DELETE CASCADE,
    foreign key (order_num) references vaccine_order(order_num) ON DELETE CASCADE);

    CREATE TRIGGER update_folks AFTER UPDATE ON folks BEGIN
  		UPDATE staff SET  S_PIN = new.PIN WHERE S_PIN = old.PIN;
			UPDATE chief SET  C_PIN = new.PIN WHERE C_PIN = old.PIN;
			UPDATE registration SET  PIN = new.PIN WHERE PIN = old.PIN;
			UPDATE residency SET  PIN = new.PIN WHERE PIN = old.PIN;
      
		END; 

    CREATE TRIGGER update_staff AFTER UPDATE ON staff BEGIN
      UPDATE modification SET  m_PIN = new.S_PIN WHERE m_PIN = old.S_PIN;
      UPDATE vaccine_order SET  PIN = new.S_PIN WHERE PIN = old.S_PIN;
    END;

    CREATE TRIGGER update_place AFTER UPDATE ON places BEGIN
      UPDATE health_center SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord;
      UPDATE residency SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord;
      UPDATE vaccine_order SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord;
      UPDATE dispatch SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord;
      UPDATE registration SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord; 
    END;

    CREATE TRIGGER update_health_center AFTER UPDATE ON health_center BEGIN
      UPDATE vaccine_order SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord;
      UPDATE dispatch SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord;
      UPDATE registration SET  x_cord = new.x_cord, y_cord = new.y_cord WHERE y_cord = old.y_cord and x_cord = y_cord; 
    END;


    CREATE TRIGGER new_registration AFTER INSERT ON registration BEGIN
      UPDATE health_center SET  curr_size = (select count(PIN) from registration where health_center.x_cord = registration.x_cord AND health_center.y_cord = registration.y_cord) WHERE 
      health_center.x_cord=x_cord and health_center.y_cord = y_cord;
      UPDATE vaccine_order SET num_vacinne = (SELECT curr_size from health_center WHERE health_center.x_cord = vaccine_order.x_cord AND health_center.y_cord = vaccine_order.y_cord) WHERE 
      vaccine_order.x_cord=x_cord AND vaccine_order.y_cord = y_cord;
    END;

    CREATE TRIGGER insert_mod AFTER UPDATE ON vaccine_order BEGIN
      INSERT INTO modification (mod_timestamp,m_PIN,order_num)values (datetime('now'),8718892831442613,new.order_num);
    
    END;

    CREATE INDEX index_folks ON folks(first_name,last_name);
 """)



In [ ]:

cursor.executescript("""
		

		delete from folks;
		delete from chief;
		delete from staff;
		delete from places;
		delete from health_center;
		delete from residency;
		delete from registration;
		delete from dispatch;
		delete from modification;
		delete from vaccine_order;

		insert into folks values (8718892831442613,'Jin-Woo','Sung','1990-10-02',9634420282,1491675758,4023244311,'["ShadowMonarch@solo.manhwa"]');
		insert into folks values (7641563892621998,'Yeon-Woo','Cha','1994-11-05',5250596029,2080628944,5468156963,'["chaY2nd@slr.manhwa"]');
		insert into folks values (7474862335619533,'Jeon-Woo','Cha','1994-05-11',7429378598,6363143632,9757867003,'["chaY1st@slr.manhwa"]');
		insert into folks values (4953499291449932,'Hae-In','Cha','2000-02-29',3631645069,6200037951,2453910037,'["chacha@solo.manhwa"]');
		insert into folks values (6708015069552521,'Yeo-Woon','Cheon','1990-01-11',8982307551,7866558277,7746596547,'["yeo@nm.manhwa"]');
		insert into folks values (3673594352062092,'Mun','Ku','1991-01-15',4218582070,6322800783,4158609493,'["ku@nm.manhwa"]');
		insert into folks values (8164679740878734,'Florentia','Lombardi','1991-07-07',8894492760,8764707276,9100576041,'["matriarch@ismtf.manhwa"]');
		insert into folks values (9448508654753811,'Perez Brivacheu','Durelli','1991-08-28',1160134390,5398208359,8089194136,'["crownPrince@ismtf.manhwa"]');
		insert into folks values (6555023229037065,'Hae-Won','Tang','1994-09-29',9876781922,5539458702,5047221054,'["masterTang@mvti.manhwa"]');
		insert into folks values (7940937800426551,'Seo-Cheon','Joo','1990-12-24',2271894014,4751187597,6715769167,'["swordDragon@mhs.manhwa","PhantomDemon@mhs.manhwa"]');
		insert into folks values (6469128022378393,'Jeong-Mok','Yu','1988-05-17',8957066088,9347957314,6452792062,'["master@mhs.manhwa"]');
		insert into folks values (4981840260436699,'Hye','Tang','1989-06-06',7486245317,6019398201,2475885742,'["tangtang@mhs.manhwa"]');
		insert into folks values (9555023229037065,'Hye','Tang','1989-06-06',8486245317,9019398201,3475885742,'["hyetang@mhs.manha"]');
		
		insert into staff values(8718892831442613);
		insert into staff values(7474862335619533);
		insert into staff values(4953499291449932);
		insert into staff values(4981840260436699);
		insert into staff values(9448508654753811);
		insert into staff values(6708015069552521);
		insert into staff values(6555023229037065);

		insert into chief values(8718892831442613);

		insert into places values(253,'Norma Ports','wonder','baltimore',21215,8,10);
		insert into places values(716 ,'Paucek Manors','wonder','baltimore',21218,10,10);
		insert into places values(508,'Eula Points','wonder','baltimore',21216,4,1);
		insert into places values(377,'Barton Gateway','land','DC',21008,1,4);
		insert into places values(78,'Alexie Road','land','DC',21008,110,112);
		insert into places values(87,'Alexie Road','land','DC',21008,118,118);
		insert into places values(77,'cityLand','Megapolis','DC',21008,118,121);

		insert into health_center values('2022-08-08',4,1,40,0);
		insert into health_center values('2022-08-10',1,4,30,0);
		insert into health_center values('2022-07-19',118,118,12,0);

		insert into residency values (8,10,8718892831442613);
		insert into residency values (8,10,7641563892621998);
		insert into residency values (8,10,7474862335619533);
		insert into residency values (8,10,4953499291449932);
		insert into residency values (8,10,6708015069552521);
		insert into residency values (8,10,9555023229037065);
		insert into residency values (10,10,3673594352062092);
		insert into residency values (10,10,8164679740878734);
		insert into residency values (110,112,9448508654753811);
		insert into residency values (110,112,6555023229037065);
		insert into residency values (110,112,7940937800426551);
		insert into residency values (110,112,6469128022378393);
		insert into residency values (110,112,4981840260436699);


		insert into registration values (8718892831442613,'2022-07-19',118,118);
		insert into registration values (4953499291449932,'2022-07-19',118,118);
		insert into registration values (4981840260436699,'2022-07-19',118,118);
		insert into registration values (3673594352062092,'2022-08-10',1,4);
		insert into registration values (7641563892621998,'2022-08-10',1,4);
		insert into registration values (8164679740878734,'2022-08-10',1,4);
		insert into registration values (7940937800426551,'2022-08-10',1,4);
		insert into registration values (6469128022378393,'2022-08-10',1,4);
		insert into registration values (9448508654753811,'2022-08-08',4,1);
		insert into registration values (7474862335619533,'2022-08-08',4,1);
		insert into registration values (6708015069552521,'2022-08-08',4,1);
		insert into registration values (9555023229037065,'2022-08-08',4,1);

		insert into dispatch values('2022-08-08', '10:00:00','15:00:00',6555023229037065,4,1,8718892831442613);
		insert into dispatch values('2022-08-08','10:00:00','15:00:00',4981840260436699,4,1,8718892831442613);
		insert into dispatch values('2022-08-08', '10:00:00','15:00:00',4953499291449932,4,1,8718892831442613);
		insert into dispatch values('2022-07-19', '14:00:00','14:30:00',7474862335619533,118,118,8718892831442613);
		insert into dispatch values('2022-07-19','14:00:00','14:30:00',6708015069552521,118,118,8718892831442613);
		insert into dispatch values('2022-07-19','14:00:00','14:30:00',6555023229037065,118,118,8718892831442613);
		insert into dispatch values('2022-08-10','12:00:00','14:00:00',9448508654753811,1,4,8718892831442613);
		insert into dispatch values('2022-08-10','12:00:00','15:00:00',6555023229037065,1,4,8718892831442613);
		insert into dispatch values('2022-08-10','12:00:00','15:00:00',4953499291449932,1,4,8718892831442613);

		insert into modification values ('2021-12-15 06:00:00',8718892831442613,125);
		insert into modification values ('2021-12-15 06:00:00',8718892831442613,126);
		insert into modification values ('2021-12-15 06:00:00',8718892831442613,127);

		insert into vaccine_order values(8718892831442613,125,3,'2020-12-15','2022-07-19',118,118);
		insert into vaccine_order values(8718892831442613,126,5,'2020-12-15','2022-08-10',1,4);
		insert into vaccine_order values(8718892831442613,127,3,'2020-12-15','2022-08-08',4,1);



""")



In [ ]:
cursor.execute("select * from vaccine_order;")
pizz = cursor.fetchall()
print(pizz)

[(8718892831442613, 125, 3, '2020-12-15', '2022-07-19', 118, 118), (8718892831442613, 126, 5, '2020-12-15', '2022-08-10', 1, 4), (8718892831442613, 127, 3, '2020-12-15', '2022-08-08', 4, 1)]


1. staff can enter the information for a new health center.
2. staff can modify their personal identifier of an existing individual.
3. folks can register for their vaccination (ensure ACID).
4. staff can delete an existing health center (and any other associated information
automatically).
5. staff can update the coordinates of a place

In [ ]:
def newHealthCenter():
  isExist = input("Is this a place existing the the database (N/Y)?")
  street_number = 0
  name = ""
  city = ""
  zip = 0
  cap = 0 
  x_cord = 0 
  y_cord = 0
  if(isExist == 'N'):
    street_number = input("what is the street number: ")
    name = input("what is the street name: ")
    city = input("What city is it located in: ")
    state = input("what is the state: ")
    zipCode = input("what is the zipcode: ")
  x_cord = input("What is the X cordinate? ")
  y_cord = input("What is the y cordinate? ")
  date = input("what date is this health center open? (year-MM-DD)")
  cap = input("What is the max capacity? ")
  
  conn.isolation_level = "DEFERRED"
  if(isExist == 'N'):
    try:
      cursor.execute("PRAGMA foreign_keys=ON")
      conn.commit()
      cursor.execute("INSERT INTO places('street_number','name','city','state','zipCode','x_cord','y_cord') VALUES (?,?,?,?,?,?,?)", (street_number,name,city,state,zipCode,x_cord,y_cord))
      cursor.execute("INSERT INTO health_center(hc_date,x_cord,y_cord,max_capacity,curr_size)VALUES (?,?,?,?,?)",(date,x_cord,y_cord,cap,0))
      conn.commit()
      print("Success")
    except sqlite3.Error as error:
      print("UNABLE TO ADD NEW PLACE AND HEALTH CENTER: ", error)
      conn.rollback()

  else:
    try:
      cursor.execute("PRAGMA foreign_keys=ON")
      conn.commit()
      cursor.execute("INSERT INTO health_center(hc_date,x_cord,y_cord,max_capacity,curr_size)VALUES (?,?,?,?,?)",(date,x_cord,y_cord,cap,0))
      conn.commit()
      print("Success")
    except sqlite3.Error as error:
      print("UNABLE TO ADD NEW HEALTH CENTER: ", error)
      conn.rollback()
    finally:
      conn.isolation_level = None
    

In [ ]:
def mod_identifier():
  oldPin = input("What is the pin that you want to update: ")
  newPin = input("What is the new pin(16 digits): ")
  conn.isolation_level = "DEFERRED"
  try:
    cursor.execute("PRAGMA foreign_keys=ON")
    conn.commit()
    cursor.execute("UPDATE folks SET PIN = ? WHERE PIN = ?", (newPin,oldPin))
    conn.commit()
    print("Success")
  except sqlite3.Error as error:
    print("UNABLE TO UPDATE FOLKS ", error)
    conn.rollback()
  finally:
      conn.isolation_level = None

In [ ]:
def delete_health_center():
  x = input("what is the x cordinate: ")
  y = input("what is the y cordinate: ")

  conn.isolation_level = "DEFERRED"
  try:
    cursor.execute("PRAGMA foreign_keys = ON;");
    conn.commit()
    cursor.execute("DELETE FROM health_center WHERE x_cord = ? and y_cord = ?", (x,y))
    conn.commit()
    print("Deleted all health_center with cordinates: ",x,",",y)
  except sqlite3.Error as error:
    print("UNABLE TO DELETE ", error)
    conn.rollback()
  finally:
      conn.isolation_level = None

In [ ]:
def update_cordinates():
  oldx = input("what is the old x cordinate: ")
  oldy= input("what is the old y cordinate: ")
  newx = input("what is the new x cordinate: ")
  newy= input("what is the new y cordinate: ")
  conn.isolation_level = "DEFERRED"
  try:
    cursor.execute("UPDATE places SET x_cord = ?, y_cord = ? WHERE x_cord = ? and y_cord = ? ", (newx,newy,oldx,oldy))
    conn.commit()
  except sqlite3.Error as error:
    print("UNABLE TO UPDATE ", error)
    conn.rollback()
  finally:
      conn.isolation_level = None
  

In [ ]:
def register():
  tester = 0

  print("This is the list of open health centers: ")
  cursor.execute("SELECT hc_date,x_cord,y_cord,street_number,name,city,state,zipcode FROM health_center NATURAL JOIN places WHERE curr_size < max_capacity")
  rows = cursor.fetchall()
  for row in rows:
    print("Date: ", row[0], "Address: ", row[3], row[4],row[5],row[6],row[7], "(" ,row[1], ",",row[2], ")")

 
  x = input("Enter the X coordinate of the health center of your choice: ")
  y = input("Enter the Y coordinate of the health center of your choice: ")
  conn.isolation_level = "DEFERRED"
  try:
    cursor.execute("SELECT street_number,name,city,state,zipcode,x_cord,y_cord FROM health_center NATURAL JOIN places WHERE curr_size < max_capacity and x_cord = ? and y_cord = ?",(x,y))
    conn.commit()
    addr = cursor.fetchall()
    if(len(addr) > 0):
      print(addr)
      print("The address for your cordinate and location is:",addr[0][0],addr[0][1], addr[0][2],addr[0][3],addr[0][4], "(" ,addr[0][5], ",",addr[0][6], ")")
    else:
      tester = 1
      print("No address was selected.")
  except sqlite3.Error as error:
    print("UNABLE TO FIND ADDRESS ", error)
    conn.rollback()
  finally:
      conn.isolation_level = None
  
  if tester == 0:
    conn.isolation_level = "DEFERRED"
    try:
      date = input("what date do you want? ")
      pin = input("what is your PIN (16 digit): ")
      cursor.execute("PRAGMA foreign_keys=ON")
      conn.commit()
      cursor.execute("insert into registration(PIN,r_date,x_cord,y_cord) values(?,?,?,?)",(pin,date,x,y))
      conn.commit()
    except sqlite3.Error as error:
      print("UNABLE TO REGISTER", error)
      conn.rollback()
    finally:
        conn.isolation_level = None



In [ ]:
# to varify user
pin = input("Enter your pin: ")
staff = []
folk = []

cursor.execute("select * from staff where S_PIN = ?",[pin])
staff = cursor.fetchall()
if(len(staff) == 0):
  cursor.execute("select * from folks where PIN = ?",[pin])
  folk = cursor.fetchall()

if(len(staff) > 0):
  print("1. Enter the information for a new health center.")
  print("2. Modify their personal identifier of an existing individual.")
  print("3. Delete an existing health center")
  print("4. Update the coordinates of a place")
  print("5. Register for your vaccination")
  choice = input("What do you want to do, chose a number")
  if choice == '1':
    newHealthCenter()
  elif choice == '2':
    mod_identifier()
  elif choice == '3':
    delete_health_center()
  elif choice == '4':
    update_cordinates()
  elif choice == '5':  
    register()
elif(len(folk) > 0):
  print("Registering for your vaccination")
  register()

conn.close()

Enter your pin: 8718892831442613
1. Enter the information for a new health center.
2. Modify their personal identifier of an existing individual.
3. Delete an existing health center
4. Update the coordinates of a place
5. Register for your vaccination
What do you want to do, chose a number5
This is the list of open health centers: 
Date:  2022-08-08 Address:  508 Eula Points wonder baltimore 21216 ( 4 , 1 )
Date:  2022-08-10 Address:  377 Barton Gateway land DC 21008 ( 1 , 4 )
Date:  2022-07-19 Address:  87 Alexie Road land DC 21008 ( 118 , 118 )
Enter the X coordinate of the health center of your choice: 4
Enter the Y coordinate of the health center of your choice: 1
[(508, 'Eula Points', 'wonder', 'baltimore', 21216, 4, 1)]
The address for your cordinate and location is: 508 Eula Points wonder baltimore 21216 ( 4 , 1 )
what date do you want? 2022-08-08
what is your PIN (16 digit): 8718892831442613


In [ ]:
%load_ext sql
connection_url = 'sqlite:///mydatabase.db'
%sql {connection_url}

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @mydatabase.db'

In [ ]:
%%sql
select * from registration;

 * sqlite:///mydatabase.db
Done.


PIN,r_date,x_cord,y_cord
8718892831442613,2022-07-19,118,118
4953499291449932,2022-07-19,118,118
4981840260436699,2022-07-19,118,118
3673594352062092,2022-08-10,1,4
7641563892621998,2022-08-10,1,4
8164679740878734,2022-08-10,1,4
7940937800426551,2022-08-10,1,4
6469128022378393,2022-08-10,1,4
9448508654753811,2022-08-08,4,1
7474862335619533,2022-08-08,4,1


In [ ]:
%%sql
drop index index_folks;
drop table if exists dispatch;
drop table if exists modification;
drop table if exists vaccine_order;
drop table if exists chief;
drop table if exists staff;
drop table if exists Registration;
drop table if exists residency;
drop table if exists folks;
drop table if exists health_center;
drop table if exists places;
drop trigger if exists update_staff;
drop trigger if exists update_folks;
drop trigger if exists update_place;
drop trigger if exists update_health_center;
drop trigger if exists new_registration;
drop trigger if exists insert_mod;


 * sqlite:///mydatabase.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]